# Using the PETSc mesh integration routines

This is probably better moved to become a test !

In [1]:
import underworld3 as uw
import numpy as np
import sympy

In [2]:
mesh = uw.util_mesh.UnstructuredSimplexBox( minCoords=(0.0,0.0), 
                                               maxCoords=(1.0,1.0), 
                                               cellSize=1.0/32.0, 
                                               regular=True)

x = mesh.N.x
y = mesh.N.y
z = mesh.N.z

I = uw.maths.Integral(mesh, x * y)
print(I.evaluate())  # 0.25

0.25


In [3]:
mesh = uw.util_mesh.StructuredQuadBox( minCoords=(0.0,0.0), 
                                          maxCoords=(1.0,1.0), 
                                          elementRes=(32,32) )

x = mesh.N.x
y = mesh.N.y
z = mesh.N.z

I2 = uw.maths.Integral(mesh, x*y)
print(I2.evaluate()) # 0.25

0.24999999999999978


In [4]:
mesh = uw.util_mesh.StructuredQuadBox( minCoords=(0.0,0.0,0.0), 
                                          maxCoords=(1.0,1.0,1.0), 
                                          elementRes=(8,8,8) )

x = mesh.N.x
y = mesh.N.y
z = mesh.N.z

I3 = uw.maths.Integral(mesh, x*y*z)
print(I3.evaluate()) # 0.125 

0.12499999999999993


In [5]:
mesh = uw.util_mesh.UnstructuredSimplexBox( minCoords=(0.0,0.0,0.0), 
                                               maxCoords=(1.0,1.0,1.0), 
                                               cellSize=1.0/8.0, 
                                               regular=True)

x = mesh.N.x
y = mesh.N.y
z = mesh.N.z

I4 = uw.maths.Integral(mesh, x * y * z)
print(I4.evaluate())  # 0.125

0.12500000000000008


In [6]:
mesh = uw.util_mesh.Annulus(radiusInner = 0.5, radiusOuter=1.0, cellSize=0.05)

x = mesh.N.x
y = mesh.N.y
z = mesh.N.z

I5 = uw.maths.Integral(mesh, 1.0)
print(I5.evaluate()) # 3 * pi / 4 = 2.35

2.356153673987439


In [7]:
mesh = uw.util_mesh.Annulus(radiusInner = 0.0, radiusOuter=1.0, cellSize=0.05)

x = mesh.N.x
y = mesh.N.y
z = mesh.N.z

I6 = uw.maths.Integral(mesh, 1.0)
print(I6.evaluate()) # pi

3.1402907966239173


In [8]:
mesh = uw.util_mesh.SphericalShell(radiusInner = 0.5, radiusOuter=1.0, cellSize=0.2)

x = mesh.N.x
y = mesh.N.y
z = mesh.N.z

I7 = uw.maths.Integral(mesh, 1)
print(I7.evaluate()) # 4/3 * 7/8 * pi 

3.6346942711638914


In [9]:
mesh = uw.util_mesh.SphericalShell(radiusInner = 0.0, radiusOuter=1.0, cellSize=0.2)

x = mesh.N.x
y = mesh.N.y
z = mesh.N.z

I8 = uw.maths.Integral(mesh, 1)
print(I8.evaluate())  # 4/3 * pi

4.12951899378597


In [23]:
mesh = uw.util_mesh.CubicSphere(radiusInner = 0.5, radiusOuter=1.0, numElements=30)

x = mesh.N.x
y = mesh.N.y
z = mesh.N.z

I9 = uw.maths.Integral(mesh, 1)
print(I9.evaluate())  # 4/3 * 7/8 * pi (3.634)

3.582770110484154


In [15]:
mesh = uw.util_mesh.SphericalShell(radiusInner = 0.0, radiusOuter=1.0, cellSize=0.5)
# mesh = uw.util_mesh.Annulus(radiusInner = 0.0, radiusOuter=1.0, cellSize=0.05)

x = mesh.N.x
y = mesh.N.y
z = mesh.N.z

radius_fn = sympy.sqrt(mesh.rvec.dot(mesh.rvec)) # normalise by outer radius if not 1.0
unit_rvec = mesh.rvec / (1.0e-10+radius_fn)

r  = sympy.sqrt(x**2+y**2) 
th = sympy.atan2(y+1.0e-10,x+1.0e-10)

v_soln   = uw.mesh.MeshVariable('U', mesh, mesh.dim, degree=2 )
p_soln   = uw.mesh.MeshVariable('P', mesh, 1, degree=1 )
t_soln   = uw.mesh.MeshVariable("T", mesh, 1, degree=3 )

swarm = uw.swarm.Swarm(mesh=mesh)
v_star     = uw.swarm.SwarmVariable("Vs", swarm, mesh.dim, proxy_degree=2)
remeshed   = uw.swarm.SwarmVariable("Vw", swarm, 1, dtype='int', _proxy=False)
X_0        = uw.swarm.SwarmVariable("X0", swarm, mesh.dim, _proxy=False)

swarm.populate(fill_param=4)

I10 = uw.maths.Integral(mesh, 1)
print(I10.evaluate())

stokes = uw.systems.Stokes(mesh, 
                velocityField=v_soln, 
                pressureField=p_soln, 
                u_degree=v_soln.degree, 
                p_degree=p_soln.degree, 
                verbose=False,
                solver_name="stokes")

stokes.add_dirichlet_bc( (0.0, 0.0, 0.0), "Upper", (0,1,2))

stokes.viscosity=1.0
stokes.bodyforce= mesh.rvec * sympy.sin(th)


stokes.solve()

I10 = uw.maths.Integral(mesh, v_soln.fn.dot(v_soln.fn))
print(I10.evaluate())


4.043518823766078
  0 SNES Function norm 0.0794159 
    Residual norms for stokes_ solve.
    0 KSP Residual norm 7.852924137926e-01 
    1 KSP Residual norm 4.949266392084e-01 
    2 KSP Residual norm 6.196786011788e-02 
    3 KSP Residual norm 4.841929529845e-03 
    4 KSP Residual norm 1.205005315792e-03 
    5 KSP Residual norm 3.566536037594e-04 
  1 SNES Function norm 0.000128478 
    Residual norms for stokes_ solve.
    0 KSP Residual norm 3.558916546921e-04 
    1 KSP Residual norm 2.281459705220e-04 
    2 KSP Residual norm 9.538972788531e-05 
    3 KSP Residual norm 2.744983510683e-05 
    4 KSP Residual norm 4.442122208107e-06 
    5 KSP Residual norm 2.642860917469e-06 
    6 KSP Residual norm 1.026638108226e-06 
    7 KSP Residual norm 2.510107102775e-07 
  2 SNES Function norm 3.54653e-09 
Nonlinear stokes_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 2
7.339116836754227e-05


In [30]:
## Check this this sort of thing works OK

mesh = uw.util_mesh.SphericalShell(radiusInner = 0.0, radiusOuter=1.0, cellSize=0.1)

x = mesh.N.x
y = mesh.N.y
z = mesh.N.z

meshvar   = uw.mesh.MeshVariable("phi", mesh, 1, degree=3 )

I = uw.maths.Integral(mesh, 1)
print(I.evaluate())  # 4/3 * pi

I.fn = 0.75
print(I.evaluate())  # pi

I.fn = x
print(I.evaluate())  # should be zero

I.fn = x + y + z 
print(I.evaluate())  # should be zero

with mesh.access(meshvar):
    meshvar.data = uw.function.evaluate(x+y+z, meshvar.coords)
    
I.fn = meshvar.fn
print(I.evaluate())  # should be zero

4.174008688031283
3.1305065160234604
-7.255924064445077e-06
1.340130712999501e-05
